# Problém batohu s vícenásobnou volbou 

In [1]:
import itertools
import random
import numpy as np
import time
import math

## Utility

In [2]:
class Item:
    def __init__(self, class_id: int, item_id: int, cubic_capacity: int, cost: int) -> None:
        """
        Trida predmetu

        Atributy:
            class_id - id tridy
            item_id - id predmetu
            cubic_capacity - objem predmetu
            cost - cena predmetu  
        """
        self.class_id = class_id
        self.item_id = item_id
        self.cubic_capacity = cubic_capacity
        self.cost = cost

    def print(self):
        print("Class-ID: %d; Item-ID: %d; Capacity: %d; Cost: %d" % (self.class_id, self.item_id, self.cubic_capacity, self.cost))

In [3]:
def generateItems(num_classes: int, num_items: int, cubic_capacity_func: int, cost_func) -> list[Item]:
    """
    Vygeneruje seznam predmetu

    Parametry:
        num_classes - pocet trid
        num_items - pocet predmetu ve tridach
        cubic_capacity_func - funkce pro generovani nahodneho objemu predmetu
        cost_func - funkce pro generovani nahodnehe ceny predmetu  
    
    Return:
        Seznam vsech vygenerovanych predmetu
    """
    items: list[Item] = []
    for class_id in range(1, num_classes+1):
        for item_id in range(1, num_items+1):
            items.append(Item(class_id, item_id, cubic_capacity_func(), cost_func()))
    return items

In [4]:
def numberOfCombinations(class_count: int, items_count: int):
    """
    Vypocita pocet kombinaci

    Paramtery:
        class_count - pocet trid
        items_count - pocet predmetu na jednu tridu   
    
    Return:
        Pocet kombinaci
    """
    count = 0
    item_count = class_count * items_count
    for i in range(1, class_count + 1):
        count += math.comb(item_count, i)

    return count

In [5]:
def numberOfClasses(combination_count: int, items_count: int):
    """
    Vypocita pocet trida ktery nejblize odpovidaji poctu kombinaci predmetu

    Paramtery:
        combination_count - pocet kombinaci
        items_count - pocet predmetu na jednu tridu   
    
    Return:
        Pocet trid
    """

    class_count = 0
    diff = float('inf')
    last_diff = diff

    for j in range(2, 30):

        count = 0
        item_count = items_count * j
        for i in range(1, j + 1):
            count += math.comb(item_count, i)

        d = abs(count - combination_count)
        if d > last_diff:
            break
        if d < diff:
            diff = d
            class_count = j   
        last_diff = d

    return class_count

In [6]:
def check_classes(combination: tuple[Item], class_count: int) -> bool:
    """
    Overi zda v kombinaci predmetu je kazdy predmet z prave jedne tridy

    Parametry:
        combination - kombinace jakymi je mozne predmety nakombinovat
        class_count - pocet trid

    Return:
        True kdyz jsou zastoupeny predmety ze vsech trid prave jednou
    """
    classes = {}
    for item in combination:
        class_id = item.class_id
        if class_id not in classes:
            classes[class_id] = 1
        else:
            return False
    return len(classes) == class_count

In [7]:
def bestSolution(items: list[Item], max_capacity: int, class_count: int, items_count: int) -> tuple([tuple[Item], float, int]):
    """
    Najde nejlepsi reseni "nejlepsi kombinace". Maximalni mozna cena & Pouze jeden predmet 
    ze tridy & Neni prekrocen maximalni objem.

    Parametry:
        items - seznam vsech predmetu
        max_capacity - maximalni kapacita batohu
        class_count - pocet trid
        items_count - pocet predmetu na jednu tridu   

    Return:
        Nejlepsi nalezna kombinace, cas potrebny pro nalezeni nejlepsiho reseni, celkovy pocet kombinaci predmetu
    """
    start = time.time()

    best_cost: int = 0
    best_combination: tuple[Item] = []

    combination_count = 0
    for i in range(1, int(len(items)/items_count)+1):
        for combination in itertools.combinations(items, i):
            combination_count +=1
            current_capacity = sum(item.cubic_capacity for item in combination)
            if current_capacity <= max_capacity:
                current_cost = sum(item.cost for item in combination)
                if current_cost > best_cost:
                    if check_classes(combination, class_count):
                        best_cost = current_cost
                        best_combination = combination


    return best_combination, float(time.time() - start), combination_count

## Nastaveni

In [8]:
# pocet generovanych trid
num_classes = 5  
# pocet predmetu v jedne tride
num_items = 3 
# objem predmetu (1 az 50)
cubic_capacity = lambda: random.randint(1, 51)
# cena predmetu
cost = lambda: random.randint(1, 51)
# maximalni kapacita batohu
max_capacity = 20 * num_classes

## Seznam vygenerovanych predmetu

In [9]:
items = generateItems(num_classes, num_items, cubic_capacity, cost)

for item in items:
    item.print()

Class-ID: 1; Item-ID: 1; Capacity: 39; Cost: 46
Class-ID: 1; Item-ID: 2; Capacity: 19; Cost: 19
Class-ID: 1; Item-ID: 3; Capacity: 4; Cost: 26
Class-ID: 2; Item-ID: 1; Capacity: 25; Cost: 30
Class-ID: 2; Item-ID: 2; Capacity: 21; Cost: 33
Class-ID: 2; Item-ID: 3; Capacity: 28; Cost: 15
Class-ID: 3; Item-ID: 1; Capacity: 7; Cost: 3
Class-ID: 3; Item-ID: 2; Capacity: 44; Cost: 41
Class-ID: 3; Item-ID: 3; Capacity: 5; Cost: 6
Class-ID: 4; Item-ID: 1; Capacity: 45; Cost: 24
Class-ID: 4; Item-ID: 2; Capacity: 51; Cost: 41
Class-ID: 4; Item-ID: 3; Capacity: 4; Cost: 43
Class-ID: 5; Item-ID: 1; Capacity: 11; Cost: 14
Class-ID: 5; Item-ID: 2; Capacity: 25; Cost: 45
Class-ID: 5; Item-ID: 3; Capacity: 45; Cost: 12


## Seznam vybranych predmetu s nejvyssi cenou

In [10]:
best, t, comb_count = bestSolution(items, max_capacity, num_classes, num_items)
print("BEST SOLUTION [max capacity: %d]:" % max_capacity)
total_cost = 0
total_capacity = 0
for item in best:
    total_cost += item.cost
    total_capacity += item.cubic_capacity
    item.print()
print("Total cost: %d; Total capacity: %d; Time: %fs" % (total_cost, total_capacity, t))
print("Combination count:", comb_count)

BEST SOLUTION [max capacity: 100]:
Class-ID: 1; Item-ID: 3; Capacity: 4; Cost: 26
Class-ID: 2; Item-ID: 2; Capacity: 21; Cost: 33
Class-ID: 3; Item-ID: 2; Capacity: 44; Cost: 41
Class-ID: 4; Item-ID: 3; Capacity: 4; Cost: 43
Class-ID: 5; Item-ID: 2; Capacity: 25; Cost: 45
Total cost: 188; Total capacity: 98; Time: 0.004099s
Combination count: 4943


## Vypocet maximalniho poctu trid, ktere je mozne vypocitat za jednu hodinu

In [11]:
# zmeri kolik casu zabere najit reseni pro 9 trid
items2 = generateItems(9, num_items, cubic_capacity, cost)
_, t2, combination_count = bestSolution(items2, max_capacity, num_classes, num_items)
print("Time for 9 classes:", t2)

# odhadne priblizny pocet kombinaci predmetu (pro 1h)
combinations_for_one_hour = int((3600.0 / t2) * combination_count)
print("Estimated number of combinations per one hour:", combinations_for_one_hour)

# vypocita pocet trid
num_classes_one_hour = numberOfClasses(combinations_for_one_hour, num_items)
print("Number of classes:", num_classes_one_hour)

Time for 9 classes: 3.963973045349121
Estimated number of combinations per one hour: 7440283388
Number of classes: 12


## Overeni

In [31]:
items_one_hour = generateItems(num_classes_one_hour, num_items, cubic_capacity, cost)
max_capacity_one_hour = 20 * num_classes_one_hour
print("Max capacity:", max_capacity_one_hour)
print("Items count:", len(items_one_hour))
print("Class count:", int(len(items_one_hour) / num_items))
min_cap = 0.0
for class_id in range(1, num_classes_one_hour + 1):
    local_min = 999
    for item in items_one_hour:
        if item.class_id == class_id:
            if local_min > item.cubic_capacity:
                local_min = item.cubic_capacity    
    min_cap += local_min

print("Minimal items capacity:", min_cap)

Max capacity: 240
Items count: 36
Class count: 12
Minimal items capacity: 134.0


In [32]:
best, t, _ = bestSolution(items_one_hour, max_capacity_one_hour, num_classes_one_hour, num_items)
print("BEST SOLUTION [max capacity: %d]:" % max_capacity_one_hour)
total_cost = 0
total_capacity = 0
for item in best:
    total_cost += item.cost
    total_capacity += item.cubic_capacity
    item.print()
print("Total cost: %d; Total capacity: %d; Time: %fs" % (total_cost, total_capacity, t))

BEST SOLUTION [max capacity: 240]:
Class-ID: 1; Item-ID: 2; Capacity: 13; Cost: 31
Class-ID: 2; Item-ID: 1; Capacity: 13; Cost: 43
Class-ID: 3; Item-ID: 1; Capacity: 5; Cost: 45
Class-ID: 4; Item-ID: 2; Capacity: 14; Cost: 37
Class-ID: 5; Item-ID: 2; Capacity: 35; Cost: 39
Class-ID: 6; Item-ID: 3; Capacity: 40; Cost: 51
Class-ID: 7; Item-ID: 2; Capacity: 5; Cost: 14
Class-ID: 8; Item-ID: 2; Capacity: 21; Cost: 23
Class-ID: 9; Item-ID: 3; Capacity: 8; Cost: 22
Class-ID: 10; Item-ID: 1; Capacity: 27; Cost: 40
Class-ID: 11; Item-ID: 1; Capacity: 48; Cost: 49
Class-ID: 12; Item-ID: 1; Capacity: 7; Cost: 29
Total cost: 423; Total capacity: 236; Time: 1422.040807s


## Zaver

In [34]:
# odhadovany cas pro reseni s "num_classes_one_hour" tridama
estimated_time_1 = numberOfCombinations(num_classes_one_hour, num_items) / combinations_for_one_hour * 60.0
print("Estimated time: %f min" % estimated_time_1)

# odhadovany cas pro reseni "num_classes_one_hour" + 1 trid
estimated_time_1 = numberOfCombinations(num_classes_one_hour + 1, num_items) / combinations_for_one_hour * 60.0
print("Estimated time: %f min" % estimated_time_1)

Estimated time: 18.078446 min
Estimated time: 118.042074 min


Za hodinu je na mem PC mozne vyresit problem s 12 tridama (3 predmety na tridu). Odhadovany cas na reseni 12 trid je 18.07 min. Vysledny cas reseni byl 1422.04 sekund (23.7 minuty). Pro reseni 13 uz by bylo potraba priblizne 2 hodin.